In [9]:
# 导入需要的Package并设定种子
suppressMessages(suppressWarnings(library(tidyverse)))
suppressWarnings(suppressMessages(library(Runuran)))
suppressWarnings(suppressMessages(library(mvtnorm)))
suppressWarnings(suppressMessages(library(e1071)))
suppressWarnings(suppressMessages(library(Bessel)))
set.seed(20250402)

In [10]:
# 生成IGN分布的随机数
rIGN <- function(n, Mu, Sigma, Lambda) {
  
  # 生成τ分布的随机数
  distr <- udig(mu = Lambda / (Lambda - 1), lambda = Lambda)
  gen <- pinvd.new(distr)
  tau <- ur(gen, n)
  
  # 随机表示方法生称多元GeN分布随机数
  z <- rmvnorm(n, rep(0, nrow(Sigma)), Sigma)
  x <- matrix(rep(Mu,n), nrow = n, byrow = TRUE) + z / sqrt(tau)
  return(x)
}

# 定义IGN分布的对数似然函数
log_f1 <- function(X, mu, sigma, lambda) {
  d <- length(mu)
  f <- function(x) {
    alpha <- t(x - mu) %*% solve(sigma) %*% (x - mu) * lambda + (lambda - 1)^2
    result <- sqrt(lambda) * exp(lambda - 1) / 2^((d-1)/2) / pi^((d+1)/2) / det(sigma)^(1/2) * besselK(sqrt(alpha), 0, expo = FALSE)
    return(result)
  }
  pdf <- apply(X, 2, f)
  return(sum(log(pdf)))
}

# IGN分布无协变量的NEM算法
IGN.NEM <- function(mu, sigma, lambda, X, ep = 1e-4) {
  index <- 0
  k <- 0
  
  n <- ncol(X)
  d <- nrow(X)
  
  f1 <- function(X, mu, sigma, lambda) {
    
    compute.alpha <- function(x) {
      return(t(x - mu) %*% solve(sigma) %*% (x - mu) + (lambda - 1)^2 / lambda)
    }
    alpha <- apply(X, 2, compute.alpha)
    beta <- lambda
    
    result1 <- BesselK(sqrt(alpha * beta), 1, expo = TRUE) / BesselK(sqrt(alpha * beta), 0, expo = TRUE) * (beta / alpha)^0.5
    result2 <- BesselK(sqrt(alpha * beta), -1, expo = TRUE) / BesselK(sqrt(alpha * beta), 0, expo = TRUE) * (beta / alpha)^(-0.5)
    return(cbind(result1, result2))
  }
  
  v1 <- rep(1, n)
  
  mu_new <- mu
  sigma_new <- sigma
  lambda_new <- lambda
  
  while (k <= 1000) {
    mu <- mu_new
    sigma <- sigma_new
    lambda <- lambda_new
    
    loglikeli <- log_f1(X, mu, sigma, lambda)
    
    a1 <- f1(X, mu, sigma, lambda)[, 1]
    mu_new <- X %*% a1 / sum(a1)
    # cat("Mu:", mu_new, "\n")
    A1 <- diag(a1)
    sigma_new <- (X - mu_new %*% v1) %*% A1 %*% t(X - mu_new %*% v1) / n
    # cat("Sigma:", sigma_new, "\n")
    Ba1 <- mean(a1)
    Bd1 <- mean(f1(X, mu, sigma, lambda)[,2])
    lambda_new <- (-1-sqrt(1-4*Ba1*(2-Ba1-Bd1))) / (2*(2-Ba1-Bd1))
    # cat("Lambda:", lambda_new, "\n")
    
    loglikeli_new <- log_f1(X, mu_new, sigma_new, lambda_new)
    # cat("Loglikelihood:", loglikeli_new, "\n")
    
    if (abs((loglikeli_new - loglikeli) / loglikeli) < ep) {
      index <- 1
      break
    }
    # cat("Iteration number:", k, "\n")
    k <- k + 1
  }
  
  result <- list(Mu = mu_new, 
                 Sigma = sigma_new, 
                 Lambda = lambda_new, 
                 Kurtosis = 3 * (lambda_new + 1) / lambda_new^2, 
                 Loglikelihood = loglikeli_new,
                 number = k, 
                 index = index)
  return(result)
}

# 设置参数并生成样本
size <- 500
Mu <- c(0, 0)
Sigma <- matrix(c(1, 0.5, 0.5, 1), nrow = 2)
Lambda <- 9
x <- rIGN(size, Mu, Sigma, Lambda)

# 矩估计确定初值
m <- colMeans(x)
s <- var(x)
kur <- apply(x, 2, kurtosis, type = 1) %>% mean()
l <- (3+sqrt(12*kur+9)) / (2*kur)

# 进行极大似然估计
IGN.NEM(m, s, l, t(x))

$Mu
            [,1]
[1,] -0.05069847
[2,]  0.04361697

$Sigma
          [,1]      [,2]
[1,] 0.8500330 0.4219935
[2,] 0.4219935 0.8796952

$Lambda
[1] 10.82328

$Kurtosis
[1] 0.3027898

$Loglikelihood
[1] -1307.324

$number
[1] 17

$index
[1] 1

In [11]:
# 生成GaN分布的随机数
rGaN <- function(n, Mu, Sigma, Lambda) {
  # 生成τ分布的随机数
  tau <- rgamma(n, Lambda, Lambda - 1)
  
  # 随机表示方法生称多元GeN分布随机数
  z <- rmvnorm(n, rep(0, nrow(Sigma)), Sigma)
  x <- matrix(rep(Mu,n), nrow = n, byrow = TRUE) + z / sqrt(tau)
  return(x)
}

# 定义GaN分布的对数似然函数
log_f2 <- function(X, mu, sigma, lambda) {
  d <- length(mu)
  f <- function(x) {
    phi <- t(x - mu) %*% solve(sigma) %*% (x - mu) / 2
    result <- gamma(lambda + 0.5) * (lambda - 1)^lambda / (2 * pi)^(d / 2) / det(sigma)^0.5 / gamma(lambda) / (lambda - 1 + phi)^(lambda + 0.5)
    return(result)
  }
  pdf <- apply(X, 2, f)
  return(sum(log(pdf)))
}

# GaN分布无协变量的NEM算法
GaN.NEM <- function(mu, sigma, lambda, X, ep = 1e-4) {
  index <- 0
  k <- 0
  
  n <- ncol(X)
  d <- nrow(X)
  
  v1 <- rep(1, n)
  
  US_Ga <- function(lambda, c) {
    index <- 0
    k <- 0
    
    lambda_new <- lambda
    
    while (index == 0) {
      lambda <- lambda_new
      q <- c + log(lambda - 1) + 1 / (lambda - 1) - digamma(lambda) + pi^2 * lambda / 6 - 1 / lambda
      lambda_new <- (q + sqrt(q^2 + 2 * pi^2 / 3)) / (pi^2 / 3)
      
      if (abs(lambda_new - lambda) < 1e-5) {
        index <- 1
        break
      }
      k <- k + 1
    }
    return(lambda_new)
  }
  
  f2 <- function(X, mu, sigma, lambda) {
    
    alpha <- lambda + 0.5
    compute.beta <- function(x) {
      return(lambda - 1 + t(x - mu) %*% solve(sigma) %*% (x - mu) / 2)
    }
    beta <- apply(X, 2, compute.beta)
    
    result1 <- alpha / beta
    result2 <- digamma(alpha) - log(beta)
    
    return(cbind(result1, result2))
  }
  
  mu_new <- mu
  sigma_new <- sigma
  lambda_new <- lambda
  
  while (k <= 1000) {
    mu <- mu_new
    sigma <- sigma_new
    lambda <- lambda_new
    
    loglikeli <- log_f2(X, mu, sigma, lambda)
    
    a2 <- f2(X, mu, sigma, lambda)[, 1]
    A2 <- diag(a2)
    mu_new <- X %*% a2 / sum(a2)
    # cat("Mu:", mu_new, "\n")
    sigma_new <- (X - mu_new %*% v1) %*% A2 %*% t(X - mu_new %*% v1) / n
    # cat("Sigma:", sigma_new, "\n")
    b2 <- f2(X, mu, sigma, lambda)[, 2]
    c <- mean(b2) - mean(a2) + 1
    lambda_new <- US_Ga(lambda, c)
    # cat("Lambda:", lambda_new, "\n")
    
    loglikeli_new <- log_f2(X, mu_new, sigma_new, lambda_new)
    # cat("Loglikelihood:", loglikeli_new, "\n")
    
    if (abs((loglikeli_new - loglikeli) / loglikeli) < ep) {
      index <- 1
      break
    }
    # cat("Iteration number:", k, "\n")
    k <- k + 1
  }
  
  
  result <- list(Mu = mu_new, 
                 Sigma = sigma_new, 
                 Lambda = lambda_new, 
                 Kurtosis = 3 / (lambda_new - 2), 
                 Loglikelihood = loglikeli_new,
                 number = k, 
                 index = index)
  return(result)
}

# 设置参数并生成样本
size <- 500
Mu <- c(0, 0)
Sigma <- matrix(c(1, 0.5, 0.5, 1), nrow = 2)
Lambda <- 10
x <- rGaN(size, Mu, Sigma, Lambda)

# 矩估计确定初值
m <- colMeans(x)
s <- var(x)
kur <- apply(x, 2, kurtosis, type = 1) %>% mean()
l <- 3 / kur + 2

# 进行极大似然估计
GaN.NEM(m, s, l, t(x))

$Mu
            [,1]
[1,] -0.06124902
[2,] -0.02680477

$Sigma
          [,1]      [,2]
[1,] 0.9227958 0.5314538
[2,] 0.5314538 0.9742821

$Lambda
[1] 10.9993

$Kurtosis
[1] 0.3333591

$Loglikelihood
[1] -1328.694

$number
[1] 11

$index
[1] 1

In [12]:
# 生成IGaN分布的随机数
rIGaN <- function(n, Mu, Sigma, Lambda) {
  
  # 生成τ分布的随机数
  random <- rgamma(n, Lambda, Lambda)
  tau <- 1 / random
  
  # 随机表示方法生称多元GeN分布随机数
  z <- rmvnorm(n, rep(0, nrow(Sigma)), Sigma)
  x <- matrix(rep(Mu,n), nrow = n, byrow = TRUE) + z / sqrt(tau)
  return(x)
}

# 定义IGaN分布的对数似然函数
log_f3 <- function(X, mu, sigma, lambda) {
  d <- length(mu)
  f <- function(x) {
    2^(-0.5*lambda+(5-2*d)/4) * lambda^(0.5*lambda+0.25)*besselK(sqrt(2*lambda*t(x-mu)%*%solve(sigma)%*%(x-mu)), -lambda+0.5,expo = FALSE) / pi^(d/2) / det(sigma)^0.5 / gamma(lambda) / (t(x-mu)%*%solve(sigma)%*%(x-mu))^(-0.5*lambda+0.25)
  }
  pdf <- apply(X, 2, f)
  return(sum(log(pdf)))
}

# IGaN分布无协变量的NEM算法
IGaN.NEM <- function(mu, sigma, lambda, X, ep = 1e-4) {
  index <- 0
  k <- 0
  
  n <- ncol(X)
  d <- nrow(X)
  
  v1 <- rep(1, n)
  
  US_IGa <- function(lambda, c) {
    index <- 0
    k <- 0
    
    lambda_new <- lambda
    
    while (index == 0) {
      lambda <- lambda_new
      q <- c + log(lambda) - digamma(lambda) + pi^2 * lambda / 6 - 1 / lambda
      lambda_new <- (q + sqrt(q^2 + 2 * pi^2 / 3)) / (pi^2 / 3)
      
      if (abs(lambda_new - lambda) < 1e-5) {
        index <- 1
        break
      }
      k <- k + 1
    }
    return(lambda_new)
  }
  
  f3 <- function(X, mu, sigma, lambda) {
    
    compute.alpha <- function(x) {
      return(t(x - mu) %*% solve(sigma) %*% (x - mu))
    }
    alpha <- apply(X, 2, compute.alpha)
    beta <- 2 * lambda
    gamma <- 0.5 - lambda
    
    
    
    result1 <- BesselK(sqrt(alpha * beta), gamma + 1, expo = TRUE) / BesselK(sqrt(alpha * beta), gamma, expo = TRUE) * (beta / alpha)^0.5
    result2 <- BesselK(sqrt(alpha * beta), gamma - 1, expo = TRUE) / BesselK(sqrt(alpha * beta), gamma, expo = TRUE) * (beta / alpha)^(-0.5)
    
    ep <- 1e-6
    result3 <- (log(BesselK(sqrt(alpha * beta), gamma + ep, expo = FALSE)) - log(BesselK(sqrt(alpha * beta), gamma, expo = FALSE))) / ep + 0.5 * log(beta / alpha)
    
    return(cbind(result1, result2, result3))
  }
  
  mu_new <- mu
  sigma_new <- sigma
  lambda_new <- lambda
  
  while (k <= 1000) {
    mu <- mu_new
    sigma <- sigma_new
    lambda <- lambda_new
    
    loglikeli <- log_f3(X, mu, sigma, lambda)
    
    a3 <- f3(X, mu, sigma, lambda)[, 1]
    A3 <- diag(a3)
    mu_new <- X %*% a3 / sum(a3)
    # cat("Mu:", mu_new, "\n")
    sigma_new <- (X - mu_new %*% v1) %*% A3 %*% t(X - mu_new %*% v1) / n
    # cat("Sigma:", sigma_new, "\n")
    
    b3 <- f3(X, mu, sigma, lambda)[, 3]
    
    d3 <- f3(X, mu, sigma, lambda)[, 2]
    c <- 1 - mean(b3) - mean(d3)
    lambda_new <- US_IGa(lambda, c)
    # cat("Lambda:", lambda_new, "\n")
    
    loglikeli_new <- log_f3(X, mu_new, sigma_new, lambda_new)
    # cat("Loglikelihood:", loglikeli_new, "\n")
    
    if (abs((loglikeli_new - loglikeli) / loglikeli) < ep) {
      index <- 1
      break
    }
    # cat("Iteration number:", k, "\n")
    k <- k + 1
  }
  
  
  result <- list(Mu = mu_new, 
                 Sigma = sigma_new, 
                 Lambda = lambda_new, 
                 Kurtosis = 3 / lambda_new, 
                 Loglikelihood = loglikeli_new,
                 number = k, 
                 index = index)
  return(result)
}

# 设定参数并生成样本
size <- 500
Mu <- c(0, 0)
Sigma <- matrix(c(1, 0.5, 0.5, 1), nrow = 2)
Lambda <- 8
x <- rIGaN(size, Mu, Sigma, Lambda)

# 矩估计确定初值
m <- colMeans(x)
s <- var(x)
kur <- apply(x, 2, kurtosis, type = 1) %>% mean()
l <- 3 / kur

# 进行极大似然估计
IGaN.NEM(m, s, l, t(x))

$Mu
             [,1]
[1,] -0.009345838
[2,]  0.002231005

$Sigma
          [,1]      [,2]
[1,] 0.9631168 0.5098799
[2,] 0.5098799 1.0455734

$Lambda
[1] 10.11452

$Kurtosis
[1] 0.2966032

$Loglikelihood
[1] -1375.417

$number
[1] 9

$index
[1] 1